In [1]:
from collections import defaultdict as ddict, OrderedDict as odict
from typing import Any, Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from rdkit.Chem import PandasTools, AllChem as Chem, Descriptors
from rdkit.Chem.Descriptors import MolWt
from sklearn.ensemble import RandomForestRegressor
import sklearn
from rdkit.Chem.rdmolops import GetFormalCharge
import torch
import deepchem as dc
import copy
from sklearn.model_selection import train_test_split
import imp
from os import listdir
from os.path import isfile, join

RDKit WARNING: [19:54:43] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
from modules.RNN import double_RNN
from modules.MPNN import double_MPNN
from modules.fit import *
from modules.transfer import transfer_weights

ModuleNotFoundError: No module named 'modules'

---
## Data preparation

In [3]:
data = pd.read_csv('data/full_pka_data.csv')
solute = data['Solute SMILES'].tolist()
solvent = data['Solvent SMILES'].tolist()
pka = data['pKa (avg)'].tolist()
data_size = len(solute)

FileNotFoundError: [Errno 2] No such file or directory: 'data/full_pka_data.csv'

In [ ]:
indices = list(range(data_size))
CV_ids, holdout_ids, _, _ = train_test_split(indices, solvent, test_size=0.2, random_state=1, stratify=solvent)
datasets = data_maker(solute, solvent, pka)

---
## Training + testing

In [ ]:
DMPNN = Model(name='DMPNN',
                model=double_MPNN(atom_messages=False),
                model_type='torch',
                data_type='SMILES')
DMPNN_att = Model(name='DMPNN with attention',
                    model=double_MPNN(atom_messages=False, interaction='exp'),
                    model_type='torch',
                    data_type='SMILES')
MPNN = Model(name='MPNN',
                model=double_MPNN(atom_messages=True),
                model_type='torch',
                data_type='SMILES')
MPNN_att = Model(name='MPNN with attention',
                    model=double_MPNN(atom_messages=True, interaction='exp'),
                    model_type='torch',
                    data_type='SMILES')
RNN = Model(name='RNN',
              model=double_RNN(interaction=None),
              model_type='torch',
              data_type='sentences')
RNN_att = Model(name='RNN with attention',
                  model=double_RNN(interaction='exp'),
                  model_type='torch',
                  data_type='sentences')
#list of all models for testing
models = [DMPNN, DMPNN_att, MPNN, MPNN_att, RNN, RNN_att]

In [4]:
trained_models = [f for f in listdir('trained/') if isfile(join('trained/', f))]

In [3]:
model_weights = []
for file in trained_models:
    if 'Gsolv' in file:
        if 'RNN_w' in file:
            model_weights.append((RNN_att,file))
        if 'DMPNN_w'
            model_weights.append((DMPNN_att,file))        
        if 'MPNN_w' in file:
            model_weights.append((DMPNN_att,file))
        if 'RNN' in file:
            model_weights.append((RNN,file))
        if 'DMPNN' in file:
            model_weights.append((DMPNN,file))
        if 'MPNN' in file:
            model_weights.append((MPNN,file))
    else:
        if 'RNN' in file:
            model_weights.append((RNN_att,file))
            model_weights.append((RNN,file))
        if 'DMPNN'
            model_weights.append((DMPNN_att,file))
            model_weights.append((DMPNN,file))
        if 'MPNN' in file:
            model_weights.append((DMPNN_att,file))
            model_weights.append((MPNN,file))

['RNN_Water_pka.pt', 'DMPNN_Gsolv.pt', 'MPNN_with_attention_Gsolv.pt', 'DMPNN_Water_pka.pt', 'MPNN_Gsolv.pt', 'RNN_Gsolv.pt', 'MPNN_Water_pka.pt', 'RNN_with_attention_Gsolv.pt', 'DMPNN_with_attention_Gsolv.pt']
